In [10]:
import pandas as pd
import pyomo.environ as pe
import math

# =====================
# DATA LOAD
# =====================
customers_df = pd.read_csv("customers2.csv")
loc_df = pd.read_csv("locations2.csv")
zbe_df = pd.read_csv("zbe_customers.csv")
params_df = pd.read_csv("params2.csv")

Q = float(params_df.loc[0, "Q"])
p = float(params_df.loc[0, "p"])
c_km = float(params_df.loc[0, "c"])
penalty = float(params_df.loc[0, "penalty"])
F = float(params_df.loc[0, "F"])

C = customers_df["customer"].astype(str).tolist()
Z = sorted(zbe_df["customer"].astype(str).tolist())
T = ["M", "A"]

d = {row["customer"]: float(row["demand"]) for _, row in customers_df.iterrows()}
a = {(row["customer"], "M"): int(row["avail_M"]) for _, row in customers_df.iterrows()}
a.update({(row["customer"], "A"): int(row["avail_A"]) for _, row in customers_df.iterrows()})

coords = {str(row["node"]): (row["x"], row["y"]) for _, row in loc_df.iterrows()}
N = list(coords.keys())

dist = {}
for i in N:
    for j in N:
        if i != j:
            xi, yi = coords[i]
            xj, yj = coords[j]
            dist[(i, j)] = math.hypot(xi - xj, yi - yj)

# =====================
# MODEL
# =====================
m = pe.ConcreteModel()

m.C = pe.Set(initialize=C)
m.Z = pe.Set(initialize=Z)
m.T = pe.Set(initialize=T)
m.N = pe.Set(initialize=N)

m.x = pe.Var(m.N, m.N, m.T, within=pe.Binary)
m.served = pe.Var(m.C, m.T, within=pe.Binary)
m.u = pe.Var(m.C, m.T, within=pe.NonNegativeReals)
m.y = pe.Var(within=pe.Binary)

def obj_rule(mm):
    revenue = sum(p * mm.served[i, t] * d[i] for i in mm.C for t in mm.T)
    travel = sum(c_km * dist[(i, j)] * mm.x[i, j, t]
                 for i in mm.N for j in mm.N if i != j for t in mm.T)
    penalty_cost = sum(penalty * d[i] * (1 - sum(mm.served[i, t] for t in mm.T))
                       for i in mm.Z)
    daily_profit = revenue - travel - penalty_cost
    return 365 * daily_profit - F * mm.y

m.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)

# =====================
# CONSTRAINTS
# =====================
def serve_once(mm, i):
    return sum(mm.served[i, t] for t in mm.T) <= 1
m.serve_once = pe.Constraint(m.C, rule=serve_once)

def avail(mm, i, t):
    return mm.served[i, t] <= a[(i, t)]
m.avail = pe.Constraint(m.C, m.T, rule=avail)

def zbe_access(mm, i):
    return sum(mm.served[i, t] for t in mm.T) <= mm.y
m.zbe_access = pe.Constraint(m.Z, rule=zbe_access)

def out_flow(mm, i, t):
    return sum(mm.x[i, j, t] for j in mm.N if j != i) == mm.served[i, t]
m.out_flow = pe.Constraint(m.C, m.T, rule=out_flow)

def in_flow(mm, i, t):
    return sum(mm.x[j, i, t] for j in mm.N if j != i) == mm.served[i, t]
m.in_flow = pe.Constraint(m.C, m.T, rule=in_flow)

def dep_out(mm, t):
    return sum(mm.x["0", j, t] for j in mm.C) == 1
m.dep_out = pe.Constraint(m.T, rule=dep_out)

def dep_in(mm, t):
    return sum(mm.x[i, "0", t] for i in mm.C) == 1
m.dep_in = pe.Constraint(m.T, rule=dep_in)

def cap(mm, t):
    return sum(d[i] * mm.served[i, t] for i in mm.C) <= Q
m.cap = pe.Constraint(m.T, rule=cap)

M = len(C)

def mtz(mm, i, j, t):
    if i == j:
        return pe.Constraint.Skip
    return mm.u[i, t] - mm.u[j, t] + M * mm.x[i, j, t] <= M - 1
m.mtz = pe.Constraint(m.C, m.C, m.T, rule=mtz)

def u_bounds(mm, i, t):
    return mm.u[i, t] <= M * mm.served[i, t]
m.u_bounds = pe.Constraint(m.C, m.T, rule=u_bounds)

# =====================
# ROUTE HELPERS
# =====================
def extract_route(turn):
    nxt = {}
    for i in N:
        for j in N:
            if i != j and pe.value(m.x[i, j, turn]) > 0.5:
                nxt[i] = j
                break
    route = ["0"]
    while route[-1] in nxt:
        route.append(nxt[route[-1]])
        if route[-1] == "0":
            break
    return route

def route_distance(route):
    return sum(dist[(route[i], route[i + 1])] for i in range(len(route) - 1))

# =====================
# SOLVE + PRINT (SCENARIOS)
# =====================
solver = pe.SolverFactory("gurobi")

def solve_and_print(label):
    res = solver.solve(m, tee=False)
    if res.solver.termination_condition != pe.TerminationCondition.optimal:
        print("Modelo no óptimo:", res.solver.termination_condition)
        return None

    revenue = sum(p * d[i] * pe.value(m.served[i, t]) for i in C for t in T)

    km_cost = sum(c_km * dist[(i, j)] * pe.value(m.x[i, j, t])
                  for i in N for j in N if i != j for t in T)

    penalty_cost = sum(penalty * d[i] *
                       (1 - sum(pe.value(m.served[i, t]) for t in T))
                       for i in Z)

    daily_profit = revenue - km_cost - penalty_cost
    annual_profit = 365 * daily_profit - F * pe.value(m.y)

    print(f"\n==== MODELO 2 — {label} ====")
    print(f"Comprar furgoneta ECO (y): {int(pe.value(m.y))}\n")

    print("---- RESULTADOS DIARIOS ----")
    print(f"Ingresos diarios:        {revenue:,.2f} €")
    print(f"Coste km diario:         {km_cost:,.2f} €")
    print(f"Penalización ZBE diaria: {penalty_cost:,.2f} €")
    print(f"Beneficio diario:        {daily_profit:,.2f} €\n")

    print("---- RESULTADOS ANUALES ----")
    print(f"Beneficio anual (operación): {365 * daily_profit:,.2f} €")
    print(f"Coste inversión ECO:         {(F if pe.value(m.y) > 0.5 else 0):,.2f} €")
    print(f"Beneficio anual neto:        {annual_profit:,.2f} €")

    for t in T:
        served_list = [i for i in C if pe.value(m.served[i, t]) > 0.5]
        load = sum(d[i] for i in served_list)
        route = extract_route(t)
        km = route_distance(route)

        print(f"\nTurno {t}:")
        print(f"  Clientes: {', '.join(served_list) if served_list else '-'}")
        print(f"  Carga: {load:.1f} / {Q}")
        print(f"  Ruta: {' -> '.join(route)}")
        print(f"  Distancia: {km:.2f} km")

    return annual_profit

# Scenario 0: no ECO
m.y.fix(0)
B0 = solve_and_print("ESCENARIO SIN FURGONETA ECO")

# Scenario 1: ECO
m.y.fix(1)
B1 = solve_and_print("ESCENARIO CON FURGONETA ECO")

# Comparison
print("\n==== COMPARACIÓN FINAL ====")
print(f"Beneficio anual SIN ECO: {B0:,.2f} €")
print(f"Beneficio anual CON ECO: {B1:,.2f} €")
print(f"Diferencia anual:        {B1 - B0:,.2f} €")

if (B1 is not None) and (B0 is not None) and (B1 > B0):
    payback = F / (B1 - B0) if (B1 - B0) > 1e-9 else float("inf")
    print(f"Payback estimado:        {payback:.2f} años")
else:
    print("La inversión NO es rentable según el modelo.")



==== MODELO 2 — ESCENARIO SIN FURGONETA ECO ====
Comprar furgoneta ECO (y): 0

---- RESULTADOS DIARIOS ----
Ingresos diarios:        1,060.00 €
Coste km diario:         56.15 €
Penalización ZBE diaria: 207.00 €
Beneficio diario:        796.85 €

---- RESULTADOS ANUALES ----
Beneficio anual (operación): 290,848.44 €
Coste inversión ECO:         0.00 €
Beneficio anual neto:        290,848.44 €

Turno M:
  Clientes: C1, C2, C4, C5, C6, C8, C9, C10, C12, C25, C28
  Carga: 102.0 / 180.0
  Ruta: 0 -> C8 -> C10 -> C28 -> C5 -> C12 -> C1 -> C9 -> C4 -> C2 -> C25 -> C6 -> 0
  Distancia: 70.24 km

Turno A:
  Clientes: C13, C14, C16, C17, C18, C20, C21, C22, C23, C26, C27
  Carga: 110.0 / 180.0
  Ruta: 0 -> C18 -> C20 -> C23 -> C14 -> C21 -> C26 -> C17 -> C16 -> C27 -> C13 -> C22 -> 0
  Distancia: 70.14 km

==== MODELO 2 — ESCENARIO CON FURGONETA ECO ====
Comprar furgoneta ECO (y): 1

---- RESULTADOS DIARIOS ----
Ingresos diarios:        1,405.00 €
Coste km diario:         61.73 €
Penalización Z